In [4]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.sparse import csr_matrix, csc_matrix, kron, eye, diags

In [ ]:
# funtion that uses GGRF
def vector_rf(W, d, f_vec, p_h, node, random_walks = 100, h = 100):
    '''
    This funtion computes the feature vector of a node using GGRF
    Args:
        W: Adjacency matrix
        d: Degree vector
        f_vec: Function to compute modulation of the random walk
        p_h: Probability of stopping the random walk
        node: Node of interest
        random_walks: Number of random walks
        h: Default value
    Returns:
        phi: Feature vector of the node
    '''
    # Initial values
    n = h
    phi = np.zeros(len(d))
    m = random_walks
    f_m = f_vec(n)

    for w in range(m):
        # Initial values for the random walk
        load = 1
        current_node = node
        terminated = False
        walk_lenght = 0
        
        # Register of the nodes visited
        register = [current_node]
        #print(phi[current_node])
        counter = 0
        while terminated == False:
            
            # In case we require more values of f
            if walk_lenght == n:
                #print("Requerí mas valores de f")
                n = 2 * n
                f_m = f_vec(n)
                #print(len(f_m))

            # Update the feature vector
            phi[current_node] += load * f_m[walk_lenght]
            #if walk_lenght == 3:
            # print(load * f_m[walk_lenght], phi[current_node])
            #print(phi)
            # Update the walk length
            walk_lenght += 1

            # Select the next node searching in the neighbors
            #print(current_node)
            neighbors = np.nonzero(W[current_node])[0]
            new_node = np.random.choice(neighbors)
            aux = []
            # If the node is already in the register, we search for a new one
            while new_node in register:
                aux.append(new_node)
                new_node = np.random.choice(neighbors)
                if len(aux) == len(neighbors):
                    break
            # If we tried all the neighbors, we select a random one
            if len(aux) == len(neighbors):
                new_node = np.random.choice(neighbors)

            # Update the load
            load = load * (d[current_node] / (1 - p_h))* W[current_node, new_node]
            #print(d[current_node] / (1 - p_h))
            #print(new_node)

            # Update the current node
            current_node = new_node

            # Update the register
            register.append(current_node)
            counter += 1

            # Check if the random walk is terminated
            terminated = (np.random.uniform(0,0.5) < p_h)
            if counter == 150:
                break
            #print(phi[node])
            #print(phi / m)

    return phi / m

In [25]:
A = np.array([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 3]
])

B = np.array([
    [0, 4],
    [5, 0],
    [0, 0]
])

A_sparse = csr_matrix(A, dtype=np.float32)
B_sparse = csr_matrix(B, dtype=np.float32)

In [26]:
A[2,2], A_sparse[2,2]

(3, 3.0)

In [31]:
c = np.nonzero(A_sparse[0])[0]
c

array([0], dtype=int32)

In [3]:
print("A_sparse:\n", A_sparse)
print("B_sparse:\n", B_sparse)

A_sparse:
   (0, 0)	1
  (1, 1)	2
  (2, 2)	3
B_sparse:
   (0, 1)	4
  (1, 0)	5


In [5]:
AxB_sparse = kron(A_sparse, B_sparse)
print("AxB_sparse:\n", AxB_sparse)

AxB_sparse:
   (0, 1)	4
  (1, 0)	5
  (3, 3)	8
  (4, 2)	10
  (6, 5)	12
  (7, 4)	15


In [10]:
A_sparse.shape

(3, 3)

In [21]:
for i in range(A_sparse.shape[0]):
    for j in range(A_sparse.shape[1]):
        print(A_sparse[i, j], end=" ")
    print()

1 0 0 
0 2 0 
0 0 3 


In [22]:
A_sparse @ A

array([[1, 0, 0],
       [0, 4, 0],
       [0, 0, 9]])